In [12]:
import server
import requests
import os
import random

STOPWORDS = server.controller.tokenizer.STOPWORDS

In [13]:
from server.controller.tokenizer import *
from server.models import db, Post, Tag, User, PostTag

In [14]:
os.environ['FLASK_CONFIG'] = './local/server-config-dev.yaml'
app = server.create_app()

In [15]:
models = server.models
db = server.models.db

In [16]:
with app.app_context():
    q = db.session.query(models.Tag)
    q = q.filter_by(has_explicit=True)
    tags = q.all()

tag_names = sorted([t.tag for t in tags])

In [27]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

In [28]:
title = 'deploying microservice apps in GCP with docker kubernetes'

In [29]:
tokens = server.controller.tokenizer.get_tokens(title)

In [30]:
with app.app_context():
    original_words = set(t['original'].lower() for t in tokens if t['original']!=t['output'])
    q = db.session.query(Tag)
    q = q.filter(db.func.lower(Tag.tag).in_(original_words))
    tags = q.all()
    
reverse_tag_lookup = {tag.tag.lower(): tag for tag in tags}

In [31]:
add_tokens = []
for token in tokens:
    if token['original'] in reverse_tag_lookup and token['original'] != token['output']:
        tag_obj = reverse_tag_lookup[token['original']]
        add_tokens.append(
            {
                'output': tag_obj.tag,
                'tag_id': tag_obj.id
            }
        )
        
tokens.extend(add_tokens)

In [32]:
tag_names = [t['output'] for t in tokens]

In [33]:
# search_tags = k_unique(tag_names, k=2)
search_tags = tag_names

with app.app_context():
    q = db.session.query(models.Post, db.func.count(models.Tag.tag))
    q = q.select_from(models.Post)
    q = q.join(models.Post.post_tags)
    q = q.join(models.PostTag.tag)
    q = q.filter(models.Tag.tag.in_(search_tags))
    q = q.group_by(models.Post)
#     q = q.having(db.func.count(models.Tag.tag) >= len(search_tags))
    q = q.having(db.func.count(models.Tag.tag) >= 1)
    
    results = q.all()
    output = []
    for r in results:
        post, count = r
        post_tags = [p.tag.tag.lower() for p in post.post_tags]
        upvotes = len(post.upvotes)
        post_tags.sort()
        output.append((post, count, upvotes, post_tags))

print('search:')
print('\t{}'.format(search_tags))
print('results:')
for post, count, upvotes, post_tags in output:
    msg = '\tpost: {id}, count: {count}, votes: {votes} <- {tags}'.format(
        id = str(post.id).zfill(2),
        count = count,
        tags = ', '.join(post_tags),
        votes=upvotes
    )
    print(msg)

search:
	['deploy', 'microservice', 'apps', 'in', 'GCP', 'with', 'docker', 'kubernetes', 'deploying']
results:
	post: 01, count: 1, votes: 0 <- apps, arad, aws, healthtech, jaggedness, upswell
	post: 03, count: 2, votes: 0 <- apps, diaskeuasis, docker, endotheliolytic, enigma, equidominant, healthtech, jacksonite, len, oilstone, primitively, relativization, shameface
	post: 07, count: 1, votes: 0 <- apps, aruncus, awikiwiki, condigness, expertize, healthtech, hector, inofficious, jill, predebit, stayer, underactor
	post: 08, count: 1, votes: 0 <- apps, containers, diaconicum, hippophile, jill, preoriginal, puparium, pussyfooting, recursion, scapulodynia, sponge, unbiddable
	post: 09, count: 1, votes: 0 <- aiguillesque, aurigerous, aws, bertholletia, continuingly, docker, jambeau, portulacaceous, prepositor, tutorial
	post: 11, count: 1, votes: 0 <- 0, deploying, post
	post: 12, count: 1, votes: 0 <- 1, deploying, post
	post: 13, count: 1, votes: 0 <- 2, deploying, post
	post: 14, count

In [36]:
post_ids = [1,2,3]
search_tags = ['aws','hippa']

In [60]:
with app.app_context():
    q = db.session.query()
    q = q.add_column(Tag.tag)
    q = q.add_column(db.func.count(Post.id).label('post_count'))
    q = q.select_from(Tag)
    q = q.join(PostTag).join(Post)
    q = q.filter(Post.id.in_(post_ids))
    q = q.filter(~Tag.tag.in_(search_tags))
    q = q.filter(Tag.has_explicit==True)
    q = q.group_by(Tag.tag)
    q = q.order_by('post_count DESC')
    print(str(q))    
    results = q.all()

SELECT tags.tag AS tags_tag, count(posts.id) AS post_count 
FROM tags INNER JOIN post_tag ON tags.id = post_tag.tag_id INNER JOIN posts ON posts.id = post_tag.post_id 
WHERE posts.id IN (%(id_1)s, %(id_2)s, %(id_3)s) AND tags.tag NOT IN (%(tag_1)s, %(tag_2)s) AND tags.has_explicit = true GROUP BY tags.tag ORDER BY post_count DESC


/Users/zen/Documents/school/startup-studio/weeve-api/env-jpn/lib/python3.6/site-packages/sqlalchemy/sql/compiler.py:643: SAWarning: Can't resolve label reference 'post_count DESC'; converting to text() (this warning may be suppressed after 10 occurrences)
  util.ellipses_string(element.element))


In [59]:
results    

[('healthtech', 3), ('apps', 2), ('docker', 1)]

In [44]:
results.

[(<Tag 19>, 2),
 (<Tag 21>, 3),
 (<Tag 22>, 1),
 (<Tag 23>, 1),
 (<Tag 24>, 1),
 (<Tag 26>, 1),
 (<Tag 27>, 1),
 (<Tag 28>, 1),
 (<Tag 29>, 1),
 (<Tag 30>, 1),
 (<Tag 31>, 1),
 (<Tag 32>, 1),
 (<Tag 33>, 1),
 (<Tag 34>, 1),
 (<Tag 35>, 1),
 (<Tag 36>, 1),
 (<Tag 37>, 1),
 (<Tag 38>, 1),
 (<Tag 39>, 1),
 (<Tag 40>, 1),
 (<Tag 41>, 1)]